<a href="https://colab.research.google.com/github/chop5551/dissertation_project/blob/main/dissertation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai


In [ ]:
# Импорт необходимых библиотек
import pathlib  # Библиотека для работы с путями к файлам
import textwrap  # Библиотека для форматирования текста
import json  # Библиотека для работы с JSON
import google.generativeai as genai  # Библиотека для работы с моделями Generative AI от Google
from IPython.display import display  # Библиотека для отображения результатов в Jupyter Notebook
from IPython.display import Markdown  # Библиотека для форматирования текста в Markdown
import time  # Библиотека для замера времени
import threading  # Импортируем threading для работы с потоками

# Функция для преобразования текста в Markdown с отступом
def to_markdown(text):
    text = text.replace('•', '  *')  # Заменяет пункты на маркеры
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))  # Форматирует текст в Markdown с отступом

# Используется для безопасного хранения ключа API
from google.colab import userdata  # Импортирует модуль для работы с данными пользователя в Colab
# Или используйте `os.getenv('GOOGLE_API_KEY')` для получения переменной среды.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')  # Получает ключ API из хранилища данных пользователя

# Настройка соединения с API Google Generative AI
genai.configure(api_key=GOOGLE_API_KEY)  # Устанавливает ключ API
model = genai.GenerativeModel('gemini-1.5-flash')  # Выбирает модель Gemini 1.5 Flash Latest
#model = genai.GenerativeModel('gemini-1.5-flash-latest')  # Выбирает модель Gemini 1.5 Flash Latest

# Загрузка онтологии из файла
with open('ontology.json', 'r', encoding='utf-8') as f:  # Открывает файл ontology.json на чтение с кодировкой UTF-8
    ontology = json.load(f)  # Загружает данные из файла в формате JSON

# Загрузка примеров из файла в виде текста
with open('examples.txt', 'r') as f:  # Открывает файл examples.txt на чтение
    examples = f.read()  # Считывает текст из файла

# Промпт с онтологией, примерами и инструкциями (упрощенная версия)
prompt_template = """
Ты — медицинский эксперт, способный анализировать клинические рекомендации и извлекать из них информацию о лечении заболеваний.

Твоя задача — проанализировать текст клинической рекомендации и преобразовать его в структурированный формат JSON в виде графа знаний, следуя структуре предоставленной онтологии лечения заболеваний.

Онтология:

{ontology}

Примеры:

{examples}

Проанализируй следующий текст и верни JSON-объект, соответствующий онтологии.
"""

# Загрузка текста КР из файла
with open('clinical_recommendation.txt', 'r') as f:  # Открывает файл clinical_recommendation.txt на чтение
    kr_text = f.read()  # Считывает текст из файла

# Формирование полного промпта с текстом КР
prompt = prompt_template.format(
    ontology=json.dumps(ontology, indent=4, ensure_ascii=False).encode('utf-8').decode(),  # Декодируем перед выводом
    examples=examples
)  # Форматирует промпт, вставляя онтологию и примеры
prompt += "\n" + kr_text  # Добавляет текст КР к промту


# Выполнение генерации контента с измерением времени
start_time = time.time()  # Начало замера времени
response = model.generate_content(prompt)  # Вызывает модель для генерации контента по промту
elapsed_time = time.time() - start_time  # Конец замера времени
print(f"Время выполнения: {elapsed_time:.2f} секунд")  # Выводит затраченное время

# Отображение результата
display(to_markdown(response.text))  # Отображает результат в Markdown

# Проверка на пустой ответ
if response.text:
  # Отображение результата
  display(to_markdown(response.text))  # Отображает результат в Markdown

  # Сохранение результата в текстовый файл
  with open('result.txt', 'w') as f:  # Открывает файл result.txt на запись
      f.write(response.text)  # Записывает текст ответа в файл
else:
    print("Model returned an empty response.")